In [4]:
from cs336_basics.bytepairStuff import getMerges, Tokenizer
print(list(x.to_bytes() for x in b'hello'))
#vocab, merges = getMerges("../data/TinyStoriesV2-GPT4-valid.txt", 10000, ['<|endoftext|>'])

[b'h', b'e', b'l', b'l', b'o']


In [4]:
import pickle

# Save
with open("vocab_tinyStories.pkl", "wb") as f:
    pickle.dump(vocab, f)

with open("merges_tinyStories.pkl", "wb") as f:
    pickle.dump(merges, f)


In [11]:
merges[0][0]

32